In [1]:
import tensorflow 
from tensorflow.keras.layers import Reshape
from keras.layers import Input, Embedding, Reshape, LSTM, Dense, Flatten
from keras.models import Model
import os
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

In [2]:
batch_size = 16
max_sequence_length = 384
max_word_length = 20
embedding_size = 128
lstm_units = 256
num_classes = 14
directory_train='.'
directory_test='.'



In [3]:
def process_text_files(directory):
    all_texts = []
    all_texts_extend = []
    
    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                content = re.sub(r'<start_".*?">', '', content)
                content = re.sub(r'<end_".*?">', '', content)
                txt_data = content.split()
                all_texts.append(txt_data)
                all_texts_extend.extend(txt_data)
    
    tokenizer = Tokenizer(char_level=True, lower=False)
    tokenizer.fit_on_texts(all_texts_extend)
    
    all_sequences=[]
    for i in range(len(all_texts)):
        all_sequences.append(tokenizer.texts_to_sequences(all_texts[i]))
        
    max_word_length = 20
    trunc_type = 'post'
    padding_type = 'post'
    
    # Pad each word sequence to the length of the longest word
    all_padded_sequences=[]
    for i in range(len(all_sequences)):
        padded_sequence = pad_sequences(all_sequences[i], maxlen=max_word_length, padding=padding_type, truncating=trunc_type)
        all_padded_sequences.append(padded_sequence)
    
    return all_padded_sequences, tokenizer.word_index


#   TRAIN
directory_path = directory_train
padded_sequences, word_index = process_text_files(directory_path)
padded_arrays = []
for arr in padded_sequences:
    pad_width = ((0, max_sequence_length - len(arr)), (0, 0))
    padded_arr = np.pad(arr, pad_width, mode='constant', constant_values=0)
    padded_arrays.append(padded_arr)

#  TEST
directory_path_test = directory_test
padded_sequences_test, word_index_test = process_text_files(directory_path_test)
padded_arrays_test = []
for arr in padded_sequences_test:
    pad_width = ((0, max_sequence_length - len(arr)), (0, 0))
    padded_arr = np.pad(arr, pad_width, mode='constant', constant_values=0)
    padded_arrays_test.append(padded_arr)


np.set_printoptions(threshold=np.inf)

#son array'i numpy array'ine çevir
x_train=np.array(padded_arrays)
x_test=np.array(padded_arrays_test)

print(x_train.shape)
print(x_test.shape)

np.set_printoptions(threshold=1000)

(547, 384, 20)
(547, 384, 20)


In [4]:
import os
import numpy as np

global wordsList,wordsIndextList

wordsList = []
wordsIndextList = []
max_sequence_length = 384


label_list = ["<start_\"company_name\">",
              "<end_\"company_name\">",
                  "<start_\"date\">",
                  "<end_\"date\">",
                  "<start_\"time\">",
                  "<end_\"time\">",
                  "<start_\"receipt_number\">",
                  "<end_\"receipt_number\">",
                  "<start_\"tax\">",
                  "<end_\"tax\">",
                  "<start_\"amount\">",
                  "<end_\"amount\">"]

label_list2= ["Pad","Others","B_Comp","l_Comp","B_Date","l_Date","B_Time","l_Time",
                  "B_Receipt","l_Receipt","B_Tax","l_Tax","B_Amount","l_Amount"]

 

def one_hot_encode_index_list():
    oneHotEncodedList = []
    unique_labels_len = len(sorted(set(label_list2)))
    for i in wordsIndextList:
        bitlist = list(0 for i in range(unique_labels_len))
        bitlist[i] = 1
        oneHotEncodedList.append(bitlist)

    for i in  range(max_sequence_length-len(oneHotEncodedList)):
        oneHotEncodedList.append([1,0,0,0,0,0,0,0,0,0,0,0,0,0])
    return oneHotEncodedList


def splitfunction(text:str):
    global wordsList,wordsIndextList
    wordsList = []
    wordsIndextList = []
    words = text.split()
    for word in words:
        wordsList.append(word)
        wordsIndextList.append(1)



def indexAssignment(tag,last):
    if(tag==None):
        return None
    index = label_list.index(tag)+2
    if(last == "l"):
       index +=1
    return index


def checkTag(word:str):
    for tag in label_list:
        index = word.find(tag)
        if index != -1:
            return tag    
    return None



def tagingWords():
    control = None
    counter = 0
    for word in wordsList:
        tag = checkTag(word)
        if tag != None:
            if(word.find("<start_") != -1):
                control = tag
            if(word.find("<start_") != -1 and word.find("<end_") == -1):
                wordsIndextList[counter] = indexAssignment(tag,"B")
                counter+=1
            elif(word.find("<start_") != -1 and word.find("<end_") != -1):
                wordsIndextList[counter] = indexAssignment(tag,"B")
                counter+=1
            else:
                wordsIndextList[counter] = indexAssignment(tag,"B")
                counter+=1
            if(word.find("<end_") != -1):
                control = None
        elif control !=None:
            wordsIndextList[counter] = indexAssignment(control,"l")
            counter+=1
        else:
            counter+=1  


def oneHotEncodedFunction(texts):    
    splitfunction(texts)
    tagingWords()
    return np.array(one_hot_encode_index_list())



# TRAIN
folder_path_train =  directory_train
file_list_train = os.listdir(folder_path_train)
y_train= []
for file_name in file_list_train:
    if file_name.endswith(".txt"):
        file_path = os.path.join(folder_path_train, file_name)
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            result = oneHotEncodedFunction(text)
            
            if len(y_train) == 0:
                y_train = np.array([result])
            else:
                y_train = np.concatenate((y_train, [result]), axis=0)


# TEST
folder_path_test =  directory_test
file_list_test = os.listdir(folder_path_test)
y_test= []
for file_name in file_list_test:
    if file_name.endswith(".txt"):
        file_path = os.path.join(folder_path_test, file_name)
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            result = oneHotEncodedFunction(text)
            
            if len(y_test) == 0:
                y_test = np.array([result])
            else:
                y_test = np.concatenate((y_test, [result]), axis=0)

                
print(y_train.shape)
print(y_test.shape)

(547, 384, 14)
(547, 384, 14)


In [5]:
# LSTM Modeli

input_shape = (max_sequence_length, max_word_length)
input_layer = Input(shape=(max_sequence_length,max_word_length), batch_size=16)
print(input_layer.shape)


(16, 384, 20)


In [6]:
char_embedding = Embedding(input_dim=max_word_length, output_dim=embedding_size)(input_layer)
print(char_embedding.shape)


(16, 384, 20, 128)


2024-03-13 18:40:10.815204: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-03-13 18:40:10.815236: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-03-13 18:40:10.815247: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-03-13 18:40:10.815840: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-13 18:40:10.816137: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
reshaped_embedding = tensorflow.reshape(char_embedding, (batch_size * max_sequence_length, max_word_length, embedding_size))
print(reshaped_embedding.shape)

(6144, 20, 128)


In [8]:
char_lstm, state_h, state_c = LSTM(units=lstm_units, return_sequences=True, return_state=True)(reshaped_embedding)
print(char_lstm)

char_lstm2, state_h, state_c = LSTM(units=lstm_units, return_sequences=True, return_state=True)(char_lstm, initial_state=[state_h, state_c])
print(char_lstm2)

KerasTensor(type_spec=TensorSpec(shape=(6144, 20, 256), dtype=tf.float32, name=None), name='lstm/PartitionedCall:1', description="created by layer 'lstm'")
KerasTensor(type_spec=TensorSpec(shape=(6144, 20, 256), dtype=tf.float32, name=None), name='lstm_1/PartitionedCall:1', description="created by layer 'lstm_1'")


In [9]:
reshaped = tensorflow.reshape(state_h, (batch_size,max_sequence_length, state_h.shape[-1]))
print(reshaped.shape)

(16, 384, 256)


In [10]:
word_lstm1, state_h, state_c = LSTM(units=lstm_units, return_sequences=True, return_state=True)(reshaped)
print(word_lstm1)
word_lstm2, state_h, state_c = LSTM(units=lstm_units, return_sequences=True, return_state=True)(word_lstm1, initial_state=[state_h, state_c])
print(word_lstm2)

KerasTensor(type_spec=TensorSpec(shape=(16, 384, 256), dtype=tf.float32, name=None), name='lstm_2/PartitionedCall:1', description="created by layer 'lstm_2'")
KerasTensor(type_spec=TensorSpec(shape=(16, 384, 256), dtype=tf.float32, name=None), name='lstm_3/PartitionedCall:1', description="created by layer 'lstm_3'")


In [11]:
classification_output = Dense(units=num_classes, activation='softmax')(word_lstm2)
print(word_lstm2)

KerasTensor(type_spec=TensorSpec(shape=(16, 384, 256), dtype=tf.float32, name=None), name='lstm_3/PartitionedCall:1', description="created by layer 'lstm_3'")


In [12]:
model = Model(inputs=input_layer, outputs=[classification_output, char_lstm])


In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(16, 384, 20)]              0         []                            
                                                                                                  
 embedding (Embedding)       (16, 384, 20, 128)           2560      ['input_1[0][0]']             
                                                                                                  
 tf.reshape (TFOpLambda)     (6144, 20, 128)              0         ['embedding[0][0]']           
                                                                                                  
 lstm (LSTM)                 [(6144, 20, 256),            394240    ['tf.reshape[0][0]']          
                              (6144, 256),                                                    

In [14]:
model.fit(x_train, y_train, epochs=10,batch_size=16, validation_data=(x_test, y_test))

Epoch 1/10


ValueError: in user code:

    File "/Users/talhaunal/tensorflow-test/env/lib/python3.8/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/Users/talhaunal/tensorflow-test/env/lib/python3.8/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/talhaunal/tensorflow-test/env/lib/python3.8/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/Users/talhaunal/tensorflow-test/env/lib/python3.8/site-packages/keras/src/engine/training.py", line 1081, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/talhaunal/tensorflow-test/env/lib/python3.8/site-packages/keras/src/engine/training.py", line 1139, in compute_loss
        return self.compiled_loss(
    File "/Users/talhaunal/tensorflow-test/en